In [1]:
#Sumber mamang Marco Bonzanini

import tweepy
from tweepy import OAuthHandler

access_token = '545740744-8r57ykKI8h87GRbBQLUfumBMiqvt3j73KB2L39z3'
access_secret = 'Pu5SFhlt1vZxLEo8JEyrZDksLoBZ135POuAEBUb70nj2c'
consumer_key = 'kEIPKjtxdljTM2Iooj8h31nDA'
consumer_secret = 'JTK1OV0jpEZfMsFsKUYUCuLBvm9tHHk88Y3j1XyhU4nbvLtXE2'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [2]:
from tweepy import Stream
from tweepy.streaming import StreamListener

In [ ]:
#fungsi streaming
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('kpk.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
    
#creating streaming

twitter_stream = Stream(api.auth, MyListener())

#start stream

twitter_stream.filter(track=['KPK','#KPKJemputPaksaMendag','Mendag'], is_async=True)

START HERE

In [2]:
import re

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)]\(]/\\OpP] # Mouth
    )"""

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [3]:
import json
with open('kpk.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])

In [4]:
import operator 
import json
from collections import Counter
 
fname = 'kpk.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        terms_all = [term for term in preprocess(tweet['text'])]
        count_all.update(terms_all)
    print(count_all.most_common(50))

[('.', 2737), (':', 1936), ('…', 1886), ('RT', 1849), ('#KPKJemputPaksaMendag', 1475), (',', 963), ('KPK', 599), ('@Prof_Kupi', 558), ('di', 488), ('@Kolak_VI', 477), ('!', 380), ('-', 347), ('@KPK_RI', 314), ('ini', 271), ('yg', 267), ('@NadLiviea', 265), ('?', 264), ('dan', 230), ('@hacknet__71', 215), ('@CNBLUE_9', 210), ('Mendag', 208), ('@idtodaydotco', 197), ('@bang_baret', 196), ('2', 194), ('Tagar', 193), ('tagar', 177), ('10', 176), ('ada', 175), ('Indonesia', 166), ('Sekjen', 161), ('@Berli_75', 152), ('saja', 151), ('dari', 150), ('PBNU', 148), ('ke', 146), ('Dipanggil', 138), ('3', 137), ('Trending', 137), ('untuk', 130), ('@dolirajabi4', 129), ('kalo', 128), ('#KpkJemputPaksaMendag', 126), ('sudah', 122), ('@Teh_TJEMPLUNG', 122), ('Jadi', 121), ('kalian', 117), ('Paling', 117), ('Twitter', 116), ('in', 115), ('x', 115)]


In [5]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [6]:
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]

# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)
# Count hashtags only
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs

In [7]:
fname = 'kpk.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        terms_all = [term for term in preprocess(tweet['text'])]
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        count_all.update(terms_stop)
    print(count_all.most_common(50))

[('…', 1886), ('RT', 1849), ('#KPKJemputPaksaMendag', 1475), ('KPK', 599), ('@Prof_Kupi', 558), ('di', 488), ('@Kolak_VI', 477), ('@KPK_RI', 314), ('ini', 271), ('yg', 267), ('@NadLiviea', 265), ('dan', 230), ('@hacknet__71', 215), ('@CNBLUE_9', 210), ('Mendag', 208), ('@idtodaydotco', 197), ('@bang_baret', 196), ('2', 194), ('Tagar', 193), ('tagar', 177), ('10', 176), ('ada', 175), ('Indonesia', 166), ('Sekjen', 161), ('@Berli_75', 152), ('saja', 151), ('dari', 150), ('PBNU', 148), ('ke', 146), ('Dipanggil', 138), ('3', 137), ('Trending', 137), ('untuk', 130), ('@dolirajabi4', 129), ('kalo', 128), ('#KpkJemputPaksaMendag', 126), ('sudah', 122), ('@Teh_TJEMPLUNG', 122), ('Jadi', 121), ('kalian', 117), ('Paling', 117), ('Twitter', 116), ('x', 115), ('Atas', 115), ('1', 112), ('orang', 112), ('pada', 109), ('ga', 109), ('@Cte_cute', 107), ('https://t.co/Zrh8gkNx8r', 107)]


In [20]:
import vincent
vincent.core.initialize_notebook()

word_freq = count_all.most_common(20)
labels, freq = zip(*word_freq)
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.display()
#bar.to_json('term_freq.json')

In [ ]:
bar.to_json('term_freq.json', html_out=True, html_path='chart.html')

In [ ]:
!python -m http.server 8000

In [24]:
import pandas
import json
import vincent

dates_ITAvWAL = []
# f is the file pointer to the JSON data set
fname = 'kpk.json'
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        # let's focus on hashtags only at the moment
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
        # track when the hashtag is mentioned
        if '#KPKJemputPaksaMendag' in terms_hash:
            dates_ITAvWAL.append(tweet['created_at'])
 
# a list of "1" to count the hashtags
ones = [1]*len(dates_ITAvWAL)
# the index of the series
idx = pandas.DatetimeIndex(dates_ITAvWAL)
# the actual series (at series of 1s for the moment)
ITAvWAL = pandas.Series(ones, index=idx)
 
# Resampling / bucketing
per_minute = ITAvWAL.resample('1Min').sum().fillna(0)
#print(per_minute)

In [25]:
vincent.core.initialize_notebook()
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Freq')
time_chart.display()

In [12]:
time_chart.to_json('chart_time_1.json', html_out=True, html_path='chart_time_1.html')

In [ ]:
!python -m http.server 8000

In [20]:
import pandas
import json
import vincent

dates_ITAvWAL = []
# f is the file pointer to the JSON data set
fname = 'kpk.json'
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        # let's focus on hashtags only at the moment
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
        # track when the hashtag is mentioned
        if '#KPKJemputPaksaMendag' in terms_hash:
            dates_ITAvWAL.append(tweet['created_at'])
 
# a list of "1" to count the hashtags
ones = [1]*len(dates_ITAvWAL)
# the index of the series
idx = pandas.DatetimeIndex(dates_ITAvWAL)
# the actual series (at series of 1s for the moment)
ITAvWAL = pandas.Series(ones, index=idx)
 
# Resampling / bucketing
per_minute = ITAvWAL.resample('1Min').sum().fillna(0)
#print(per_minute)

In [34]:
dates_ITAvWAL = []
# f is the file pointer to the JSON data set
fname = 'kpk.json'
with open(fname, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        # let's focus on hashtags only at the moment
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('')]
        # track when the hashtag is mentioned
        if 'Mendag' in terms_hash:
            dates_ITAvWAL.append(tweet['created_at'])
 
# a list of "1" to count the hashtags
ones = [1]*len(dates_ITAvWAL)
# the index of the series
idx = pandas.DatetimeIndex(dates_ITAvWAL)
# the actual series (at series of 1s for the moment)
ITAvWAL = pandas.Series(ones, index=idx)
 
# Resampling / bucketing
per_minute = ITAvWAL.resample('1Min').sum().fillna(0)
#print(per_minute)

In [35]:
vincent.core.initialize_notebook()
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Freq')
time_chart.display()